# How well do jobs and skills pay for Data Analysts?

#### Methodology
1. Evaluate median salary for all data jobs
2. Find median salary per skill for Data Analysts
3. Visualize for highest paying skills
4. Visualize for highest demand skills

#### Original Exploration

[18_Matplotlib_Box_Plots.ipynb](18_Matplotlib_Box_Plots.ipynb)

[19_Exercise_Skill_Pay_Analysis.ipynb](19_Exercise_Skill_Pay_Analysis.ipynb)